# Train
## Use this notebook for training new image-to-image translation models
This notebook train a new DS-CycleGAN model.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from trainer.config import config
from trainer.utils import dataset
from trainer.models import networks
from trainer import utils
from trainer import models
from trainer import callbacks

Unknown args: ['-f', '/home/jupyter/.local/share/jupyter/runtime/kernel-6e7538ad-8a5c-4471-8c76-11d1c20e1851.json']
Parsed args: {'model_dir': './trained_models', 'startLRdecay': 100, 'cycle_consistency_loss': 10, 'm': True, 'in_h': 256, 'ds_loss': 10, 'id_loss': 5, 'job_dir': 'gs://duke-bme590-cz/ds-cyclegan/tmp/1575520129.8975487', 'disc_loss': 1, 'generator_norm': 'instance', 'is_test': False, 'latent_z_dim': 8, 'ds_count': 1067, 'buffer_size': 1000, 'epochs': 10, 'discriminator_norm': 'instance', 'in_w': 256, 'bs': 1, 'ds_name': 'cycle_gan/horse2zebra'}


In [3]:
train_X, train_Y, test_X, test_Y = dataset.generate_dataset('cycle_gan/facades')
dataset_count = 400

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset cycle_gan downloaded and prepared to /home/jupyter/tensorflow_datasets/cycle_gan/horse2zebra/0.1.0. Subsequent calls will reuse this data.


In [9]:
g_AB = networks.bicycle_generator(img_shape=(256, 256, 3), z_shape=(config.latent_z_dim,), norm=config.generator_norm, skip=False, z_in='all')
g_BA = networks.bicycle_generator(img_shape=(256, 256, 3), z_shape=(config.latent_z_dim,), norm=config.generator_norm, skip=False, z_in='all')
d_A = networks.create_discriminator(shape=(256, 256, 3), norm=config.discriminator_norm)
d_B = networks.create_discriminator(shape=(256, 256, 3), norm=config.discriminator_norm)

To train classic cycleGAN without diversity regularization, use the following:

In [ ]:
# g_AB = networks.create_generator(shape=(config.in_h, config.in_w, 3), norm=config.generator_norm, skip=False)

# g_BA = networks.create_generator(shape=(config.in_h, config.in_w, 3), norm=config.generator_norm, skip=False)


In [11]:
LOG_DIR = './trainer/trained_models/user/logs'
MODEL_DIR = './trainer/trained_models/user/models'

model = models.DSGAN(shape = (None, None, 3),
                     g_AB=g_AB,
                     g_BA=g_BA,
                     d_B=d_B,
                     d_A=d_A)

model.compile(lr_g=2e-4, lr_d=2e-4,
              d_loss='mse',
              g_loss = [
                 'mse', 'mse',
                 'mae', 'mae',
                 'mae', 'mae'
              ], loss_weights = [
                 1,  1,
                 10, 10,
                 5, 5,
                 4, 4
              ],
              metrics=[utils.ssim])

def scheduler(epoch):
    if epoch < config.startLRdecay:
        return 2e-4
    else:
        epochs_passed = epoch - config.startLRdecay
        decay_step = 2e-4 / (config.epochs - config.startLRdecay)
        return 2e-4 - epochs_remaining * decay_step

# Generate Callbacks
LRscheduler = callbacks.MultiLRScheduler(scheduler, training_models=[model.d_A, model.d_B, model.combined])
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, write_graph=True, update_freq='epoch')
start_tensorboard = callbacks.StartTensorBoard(LOG_DIR)
prog_bar = tf.keras.callbacks.ProgbarLogger(count_mode='steps', stateful_metrics=None)
log_code = callbacks.LogCode(LOG_DIR, './trainer')
copy_keras = callbacks.CopyKerasModel(MODEL_DIR, LOG_DIR)
saving = callbacks.MultiModelCheckpoint(MODEL_DIR + '/model.{epoch:02d}-{val_ssim:.10f}.hdf5',
                                        monitor='val_ssim', verbose=1, freq='epoch', mode='max', save_best_only=False,
                                        save_weights_only=True,
                                        multi_models=[('g_AB', g_AB), ('g_BA', g_BA), ('d_A', d_A), ('d_B', d_B)])
image_gen = callbacks.GenerateImages(g_AB, train_X, train_Y, LOG_DIR, interval=int(dataset_count/config.bs), z_shape=(config.latent_z_dim,))

# Fit the model
model.fit(train_X, train_Y,
    batch_size=config.bs,
    steps_per_epoch=(dataset_count // config.bs),
                     epochs=100,
                     validation_data=(test_X, test_Y),
                     validation_steps=10,
                     callbacks=[log_code, tensorboard, prog_bar, image_gen, saving,
                                copy_keras, start_tensorboard])

Epoch 1/100
   2/1000 [..............................] - ETA: 12:51:33 - d_loss: 0.4502 - d_acc: 50.0000 - g_loss: 8.0664 - adv_loss: 0.8056 - recon_loss: 0.2156 - id_loss: 0.2156 - ds_loss: -0.0017

KeyboardInterrupt: 

Quickly visualize your results here

In [2]:
import matplotlib.pyplot as plt
import numpy as np
def generate_images(model, test_input):
  z1 = tf.random.normal((1, 8))
  prediction = model([test_input, z1])
    
  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i])
    plt.axis('off')
  plt.show()

In [ ]:
for inp in test_X.take(5):
  generate_images(g_AB, inp)